In [1]:
# Import packages needed & load image
import gc
import pickle
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import platform


from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io, draw
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal
import pandas as pd

import general_funcs

# OS related settings
if platform.system() == 'Windows':
    # %matplotlib nbagg
    # Sometimes tk/qt will not let cells rerun after an ERROR occurs
    # %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    ;
# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Use skimage to load multi-layer tiff file
# Tk().withdraw()
image_file = askopenfilename(title='Load image file', initialdir='./data/field_image')
img = io.imread(image_file)
print('Image File:', image_file)
print("Image Shape:", img.shape)

h, w, d = img.shape

# Extract layers from the multilayer tiff file and do some adjustments
layer_RGB, layer_IR, layer_mask = general_funcs.extract_layers(img)

ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()
lon_meter_per_pix, lat_meter_per_pix = general_funcs.meter_per_pix(gt)
r = round(200 * lon_meter_per_pix)

del(img)
# Play sound when done
general_funcs.done()

Interactive plot activated


Image File: /Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/field_image/BRC/BRC_20190904_121517_Thermal.tif
Image Shape: (3060, 14710, 2)

Done


In [2]:
# Select MTPs

def show_MTP(ax):
    mtp_ref_file = askopenfilename(title='Load MTP', initialdir='./data/mtp')
    
    try:
        with open(mtp_ref_file, 'rb') as f:
            mtp_ref = pickle.load(f)
    except Exception as e:
        showerror(type(e).__name__, str(e))
    
    for i in range(len(mtp_ref)):
        x, y = mtp_ref[i]
        ax.text(x, y, 'MTP ' + str(i+1))
        
        circle = patches.Circle([x, y], radius=r*10, color='red')
        ax.add_patch(circle)


def show_img_ref(button):
    image_file_ref = askopenfilename(title='Load image file', initialdir='./data/field_image')
    img_ref = io.imread(image_file_ref)
    ds_ref = gdal.Open(image_file_ref)
    gt_ref = ds_ref.GetGeoTransform()
    
#     plot_loc_file_ref = askopenfilename(title='Load plot location file', initialdir='./data/plot_location')
#     try:
#         with open(plot_loc_file_ref, 'rb') as f:
#             interested_area_ref = pickle.load(f)
#             plot_vertices_gps_ref = pickle.load(f)
#             plot_notes_ref = pickle.load(f)
#     except Exception as e:
#         showerror(type(e).__name__, str(e))
#     plot_vertices_ref = general_funcs.plotVGPS2plotV(plot_vertices_gps_ref, gt_ref)
    
    
    h_ref, w_ref, d_ref = img_ref.shape
    layer_RGB_ref, layer_IR_ref, layer_mask_ref = general_funcs.extract_layers(img_ref)
    del(img_ref)
    
    
    fig2, ax_ref = plt.subplots(figsize=(7, 7))
    plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)

    if d_ref == 5 or d_ref == 4:
        myax_ref = ax_ref.imshow(layer_RGB_ref)
    elif d_ref == 2:
        mask_not_0_inds_ref = np.where(layer_mask_ref > 0)
        vmin_ref, vmax_ref = general_funcs.cal_vmin_vmax(layer_IR_ref, layer_mask_ref)
        myax_ref = ax_ref.imshow(layer_IR_ref, cmap='gist_gray', vmin=vmin_ref, vmax=vmax_ref)
    
    show_MTP(ax_ref)
    

def onselect(vert):
    ;
    
    
def set_mtp(mtp, fig):
    tb = fig.canvas.toolbar
    def on_click(event):
        x, y = round(event.xdata), round(event.ydata)
        if tb.mode == '':
            mtp.append([x, y])
            ax.text(x, y, 'MTP ' + str(len(mtp)))
            
            circle = patches.Circle([x, y], radius=r, color='red')
            ax.add_patch(circle)
    return(on_click)


def reset_last_mtp(mtp, fig):
    tb = fig.canvas.toolbar
    def on_key(event):
        if event.key == 'escape' and len(ax.texts) > 0:
            ax.texts[-1].set_visible(False)
            ax.texts.pop()
            ax.patches.pop()
            mtp.pop()
    return(on_key)


def save_mtp(button):
    fn = image_file.split('/')[-1].split('.')[0]
    file_name = asksaveasfilename(filetypes=[('pickle', '*.pkl')], title='Save MTP', initialfile=fn+'_mtp', initialdir='./data/mtp')
    if not file_name:
        return
    if not file_name.endswith('.pkl'):
        file_name += '.pkl'
        
    try:
        with open(file_name, 'wb') as f:
            pickle.dump(mtp, f)
        print('GPS coordinates saved to', file_name)
    except Exception as e:
        showerror(type(e).__name__, str(e))

# Button widgets
button_layout = widgets.Layout(width='auto')
button_show_img_ref = widgets.Button(description='Open Reference Image', layout=button_layout)
button_save = widgets.Button(description='Save', layout=button_layout)
# Box widgets
box_layout = widgets.Layout(width='auto')
all_widgets = widgets.VBox(children=[button_show_img_ref, button_save], layout=box_layout)
display(all_widgets)

button_show_img_ref.on_click(show_img_ref)
button_save.on_click(save_mtp)

out = widgets.Output()
display(out)
with out:
    plt.close('all')
    fig1, ax = plt.subplots(figsize=(7, 7))
    plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)
    if d == 5 or d == 4:
        myax = ax.imshow(layer_RGB)
    elif d == 2:
        mask_not_0_inds = np.where(layer_mask > 0)
        vmin, vmax = general_funcs.cal_vmin_vmax(layer_IR, layer_mask)
        myax = ax.imshow(layer_IR, cmap='gist_gray', vmin=vmin, vmax=vmax)
    #     cbar = fig.colorbar(myax)
    ps = PolygonSelector(ax, onselect, useblit=True, lineprops=dict(linewidth=0), markerprops=dict(markersize=0, mec='r', mfc='y', alpha=0.5))
    
    mtp = []
    
    cid_set_mtp = fig1.canvas.mpl_connect('button_press_event', set_mtp(mtp, fig1))
    cid_reset_last_mtp = fig1.canvas.mpl_connect('key_press_event', reset_last_mtp(mtp, fig1))

Output()

Traceback (most recent call last):
  File "/Users/lj/opt/miniconda3/envs/hyperspect/lib/python3.9/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-2-3ddf7edd97e4>", line 62, in on_click
    x, y = round(event.xdata), round(event.ydata)
TypeError: type NoneType doesn't define __round__ method


GPS coordinates saved to /Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/mtp/BRC_20190904_121517_Thermal_mtp.pkl


Traceback (most recent call last):
  File "/Users/lj/opt/miniconda3/envs/hyperspect/lib/python3.9/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-2-3ddf7edd97e4>", line 62, in on_click
    x, y = round(event.xdata), round(event.ydata)
TypeError: type NoneType doesn't define __round__ method


In [5]:
mtp_ref_file = askopenfilename(title='Load MTP', initialdir='./data/mtp')
    
try:
    with open(mtp_ref_file, 'rb') as f:
        mtp_ref = pickle.load(f)
except Exception as e:
    showerror(type(e).__name__, str(e))

In [7]:
print(len(mtp_ref))

28
